## Imports

In [3]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Nov 17 12:01:13 2017
@author: abhijeet
"""

import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
import numpy as np
import os

Using TensorFlow backend.


## Functions

In [0]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

## Data load and preproccess

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


## Model architecture 

In [0]:
weight_decay = 1e-4

model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)       

## Data augumentation

In [0]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

## Callbacks

In [0]:
run_path = "/content/drive/My Drive/Colab Notebooks/mgu/models"
mcp_save = ModelCheckpoint(os.path.join(run_path, 'model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5'),
                           save_best_only=True, monitor='val_accuracy')
callbacks = [LearningRateScheduler(lr_schedule), mcp_save]

## Training

In [0]:
batch_size = 64
opt_rms = keras.optimizers.rmsprop(lr=0.00001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.load_weights("/content/drive/My Drive/Colab Notebooks/mgu/models/model-083-0.883371-0.891900.h5")
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,\
                    verbose=1,validation_data=(x_test,y_test), callbacks=callbacks)
#save to disk
model_json = model.to_json()
with open('/content/drive/My Drive/Colab Notebooks/mgu/models/model.json', 'w') as json_file:
    json_file.write(model_json)

In [7]:
pip install -U keras-tuner

Requirement already up-to-date: keras-tuner in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [0]:
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers


class MyHyperModel(HyperModel):

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):

        weight_decay = 1e-4

        model = keras.Sequential()
        model.add(layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(32,32,3)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.2))

        model.add(layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(0.3))

        model.add(layers.Conv2D(filters=hp.Int('filters',
                                min_value=100,
                                max_value=150,
                                step=32),
                          kernel_size=(3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
        model.add(layers.Activation('elu'))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2,2)))
        model.add(layers.Dropout(rate=hp.Choice('dropout_rate',
                          values=[1e-2, 2e-2, 3e-2])))

        model.add(layers.Flatten())
        model.add(layers.Dense(num_classes, activation='softmax'))
        opt_rms = keras.optimizers.RMSprop(lr=0.00001,decay=1e-6)
        model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
        return model


hypermodel = MyHyperModel(num_classes=10)

tuner = RandomSearch(
    hypermodel,
    objective='val_accuracy',
    max_trials=10,
    directory='/content/drive/My Drive/Colab Notebooks/mgu/hyper',
    project_name='hyper')

batch_size = 64
epochs = 60
tuner.search(datagen.flow(x_train, y_train, batch_size=batch_size),
            steps_per_epoch=int((len(x_train)-1) / batch_size), # steps per epoch
            epochs=epochs,
            validation_data=(x_test, y_test))

INFO:tensorflow:Reloading Oracle from existing project /content/drive/My Drive/Colab Notebooks/mgu/hyper/hyper/oracle.json
Epoch 1/60
781/781 [==============================] - 58s 75ms/step - loss: 2.3075 - accuracy: 0.2539 - val_loss: 1.8263 - val_accuracy: 0.3731
Epoch 2/60
781/781 [==============================] - 58s 75ms/step - loss: 1.9475 - accuracy: 0.3421 - val_loss: 1.8136 - val_accuracy: 0.3887
Epoch 3/60
781/781 [==============================] - 58s 74ms/step - loss: 1.8126 - accuracy: 0.3823 - val_loss: 1.8030 - val_accuracy: 0.3987
Epoch 4/60
781/781 [==============================] - 59s 75ms/step - loss: 1.7119 - accuracy: 0.4089 - val_loss: 1.7822 - val_accuracy: 0.4117
Epoch 5/60
781/781 [==============================] - 58s 75ms/step - loss: 1.6366 - accuracy: 0.4352 - val_loss: 1.7655 - val_accuracy: 0.4225
Epoch 6/60
781/781 [==============================] - 58s 74ms/step - loss: 1.5747 - accuracy: 0.4509 - val_loss: 1.7743 - val_accuracy: 0.4263
Epoch 7/60
78

Epoch 1/60
781/781 [==============================] - 58s 74ms/step - loss: 2.2392 - accuracy: 0.2693 - val_loss: 1.7636 - val_accuracy: 0.3924
Epoch 2/60
781/781 [==============================] - 58s 74ms/step - loss: 1.8743 - accuracy: 0.3628 - val_loss: 1.7124 - val_accuracy: 0.4183
Epoch 3/60
781/781 [==============================] - 59s 75ms/step - loss: 1.7236 - accuracy: 0.4071 - val_loss: 1.6388 - val_accuracy: 0.4512
Epoch 4/60
781/781 [==============================] - 59s 75ms/step - loss: 1.6243 - accuracy: 0.4367 - val_loss: 1.6451 - val_accuracy: 0.4663
Epoch 5/60
781/781 [==============================] - 59s 75ms/step - loss: 1.5559 - accuracy: 0.4634 - val_loss: 1.6148 - val_accuracy: 0.4737
Epoch 6/60
781/781 [==============================] - 59s 75ms/step - loss: 1.5107 - accuracy: 0.4766 - val_loss: 1.6319 - val_accuracy: 0.4742
Epoch 7/60
781/781 [==============================] - 58s 75ms/step - loss: 1.4717 - accuracy: 0.4900 - val_loss: 1.5946 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 63s 81ms/step - loss: 1.9751 - accuracy: 0.3414 - val_loss: 1.6371 - val_accuracy: 0.4355
Epoch 2/60
781/781 [==============================] - 63s 80ms/step - loss: 1.6467 - accuracy: 0.4344 - val_loss: 1.6425 - val_accuracy: 0.4442
Epoch 3/60
781/781 [==============================] - 63s 80ms/step - loss: 1.5198 - accuracy: 0.4728 - val_loss: 1.6760 - val_accuracy: 0.4553
Epoch 4/60
781/781 [==============================] - 63s 81ms/step - loss: 1.4460 - accuracy: 0.5016 - val_loss: 1.6661 - val_accuracy: 0.4630
Epoch 5/60
781/781 [==============================] - 63s 80ms/step - loss: 1.3928 - accuracy: 0.5237 - val_loss: 1.6791 - val_accuracy: 0.4713
Epoch 6/60
781/781 [==============================] - 63s 80ms/step - loss: 1.3577 - accuracy: 0.5356 - val_loss: 1.6600 - val_accuracy: 0.4738
Epoch 7/60
781/781 [==============================] - 62s 80ms/step - loss: 1.3141 - accuracy: 0.5524 - val_loss: 1.6050 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 61s 78ms/step - loss: 2.0936 - accuracy: 0.3132 - val_loss: 1.6587 - val_accuracy: 0.4348
Epoch 2/60
781/781 [==============================] - 60s 77ms/step - loss: 1.7545 - accuracy: 0.4002 - val_loss: 1.6467 - val_accuracy: 0.4398
Epoch 3/60
781/781 [==============================] - 60s 77ms/step - loss: 1.6122 - accuracy: 0.4440 - val_loss: 1.6761 - val_accuracy: 0.4503
Epoch 4/60
781/781 [==============================] - 60s 77ms/step - loss: 1.5231 - accuracy: 0.4735 - val_loss: 1.6839 - val_accuracy: 0.4620
Epoch 5/60
781/781 [==============================] - 61s 78ms/step - loss: 1.4619 - accuracy: 0.4940 - val_loss: 1.7161 - val_accuracy: 0.4588
Epoch 6/60
781/781 [==============================] - 62s 79ms/step - loss: 1.4179 - accuracy: 0.5118 - val_loss: 1.7576 - val_accuracy: 0.4583
Epoch 7/60
781/781 [==============================] - 61s 79ms/step - loss: 1.3808 - accuracy: 0.5260 - val_loss: 1.7136 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 58s 75ms/step - loss: 2.1734 - accuracy: 0.2913 - val_loss: 1.7123 - val_accuracy: 0.4222
Epoch 2/60
781/781 [==============================] - 59s 76ms/step - loss: 1.8405 - accuracy: 0.3753 - val_loss: 1.6181 - val_accuracy: 0.4501
Epoch 3/60
781/781 [==============================] - 59s 76ms/step - loss: 1.6909 - accuracy: 0.4190 - val_loss: 1.6325 - val_accuracy: 0.4522
Epoch 4/60
781/781 [==============================] - 60s 77ms/step - loss: 1.6042 - accuracy: 0.4447 - val_loss: 1.6225 - val_accuracy: 0.4584
Epoch 5/60
781/781 [==============================] - 60s 77ms/step - loss: 1.5385 - accuracy: 0.4686 - val_loss: 1.5910 - val_accuracy: 0.4733
Epoch 6/60
781/781 [==============================] - 60s 77ms/step - loss: 1.4912 - accuracy: 0.4843 - val_loss: 1.5907 - val_accuracy: 0.4773
Epoch 7/60
781/781 [==============================] - 60s 77ms/step - loss: 1.4526 - accuracy: 0.4999 - val_loss: 1.6226 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 62s 79ms/step - loss: 2.2469 - accuracy: 0.2728 - val_loss: 1.7546 - val_accuracy: 0.3962
Epoch 2/60
781/781 [==============================] - 62s 79ms/step - loss: 1.9104 - accuracy: 0.3580 - val_loss: 1.7195 - val_accuracy: 0.4153
Epoch 3/60
781/781 [==============================] - 62s 79ms/step - loss: 1.7592 - accuracy: 0.3986 - val_loss: 1.7572 - val_accuracy: 0.4170
Epoch 4/60
781/781 [==============================] - 62s 79ms/step - loss: 1.6534 - accuracy: 0.4302 - val_loss: 1.7440 - val_accuracy: 0.4290
Epoch 5/60
781/781 [==============================] - 62s 79ms/step - loss: 1.5839 - accuracy: 0.4513 - val_loss: 1.7859 - val_accuracy: 0.4310
Epoch 6/60
781/781 [==============================] - 62s 79ms/step - loss: 1.5348 - accuracy: 0.4666 - val_loss: 1.8151 - val_accuracy: 0.4298
Epoch 7/60
781/781 [==============================] - 62s 79ms/step - loss: 1.4896 - accuracy: 0.4852 - val_loss: 1.7948 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 68s 87ms/step - loss: 2.0418 - accuracy: 0.3279 - val_loss: 1.6876 - val_accuracy: 0.4288
Epoch 2/60
781/781 [==============================] - 65s 83ms/step - loss: 1.7072 - accuracy: 0.4142 - val_loss: 1.6658 - val_accuracy: 0.4509
Epoch 3/60
781/781 [==============================] - 67s 86ms/step - loss: 1.5898 - accuracy: 0.4512 - val_loss: 1.6816 - val_accuracy: 0.4526
Epoch 4/60
781/781 [==============================] - 67s 86ms/step - loss: 1.5118 - accuracy: 0.4790 - val_loss: 1.6846 - val_accuracy: 0.4588
Epoch 5/60
781/781 [==============================] - 65s 83ms/step - loss: 1.4544 - accuracy: 0.5019 - val_loss: 1.6803 - val_accuracy: 0.4598
Epoch 6/60
781/781 [==============================] - 66s 84ms/step - loss: 1.4183 - accuracy: 0.5124 - val_loss: 1.7221 - val_accuracy: 0.4647
Epoch 7/60
781/781 [==============================] - 65s 84ms/step - loss: 1.3852 - accuracy: 0.5236 - val_loss: 1.6909 - val_accuracy:

Epoch 1/60
781/781 [==============================] - 56s 72ms/step - loss: 2.1711 - accuracy: 0.2877 - val_loss: 1.6947 - val_accuracy: 0.4189
Epoch 2/60
781/781 [==============================] - 61s 79ms/step - loss: 1.8475 - accuracy: 0.3709 - val_loss: 1.6355 - val_accuracy: 0.4441
Epoch 3/60
781/781 [==============================] - 62s 80ms/step - loss: 1.7120 - accuracy: 0.4085 - val_loss: 1.5926 - val_accuracy: 0.4533
Epoch 4/60
781/781 [==============================] - 62s 80ms/step - loss: 1.6256 - accuracy: 0.4338 - val_loss: 1.6056 - val_accuracy: 0.4610
Epoch 5/60
781/781 [==============================] - 62s 79ms/step - loss: 1.5540 - accuracy: 0.4582 - val_loss: 1.5851 - val_accuracy: 0.4687
Epoch 6/60
781/781 [==============================] - 61s 78ms/step - loss: 1.5131 - accuracy: 0.4747 - val_loss: 1.6067 - val_accuracy: 0.4671
Epoch 7/60
781/781 [==============================] - 62s 79ms/step - loss: 1.4686 - accuracy: 0.4907 - val_loss: 1.6134 - val_accuracy:

In [0]:
# Show a summary of the search
tuner.results_summary()

# Retrieve the best model.
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model.
loss, accuracy = best_model.evaluate(x_test, y_test)

In [0]:
best_mode.save('/content/drive/My Drive/Colab Notebooks/mgu/hyper/best_model.h5')

## Testing

In [0]:
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))